In [1]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import time

In [2]:
df1=pd.read_csv('MyData.csv')
df1=df1[df1['Sale.Type']=='Credit']
df2=df1[['Pstng.Date','Account','Local.Crcy.Amt']]
df2.index=pd.to_datetime(df1['Pstng.Date'])
df=df2#pd.read_csv('CashData.csv',  parse_dates = ['Pstng.Date'], index_col=1)
df.drop(columns='Pstng.Date',inplace=True)
print('Total Number of Unique Customers = ', len(np.unique(df['Account'])))

Total Number of Unique Customers =  129


In [3]:
dic={}
def cash(df,date,offset_days,offset_count,c=[1,2,1,1,1]):
    unique_customer=np.unique(df['Account'])
    df=df.sort_index()
    out=unique_customer
    for cust in unique_customer[:5]:
        df_targets=df[df['Account'] == cust]
        #print(df_targets.head())
        df_targets.drop(columns='Account',inplace=True)
        train1=df_targets[df_targets.index <= pd.to_datetime(date,format='%Y%m%d') ]
        train=train1.groupby(train1.index,as_index=True).mean()
        valid=df_targets[df_targets.index >= pd.to_datetime(date,format='%Y%m%d') ]
        valid=valid.groupby(valid.index,as_index=True).mean()
        
        
        if(len(train)<2):
            print()
            print('NO TRANSACTIONS EXPECTED FROM THIS CUSTOMER ',cust)
            print()
            print('Moving on...')
            print('---------------------------------------------------------------------------------------------------')
            continue
            
        d=pd.to_datetime(date,format='%Y%m%d')-pd.to_timedelta(offset_days, unit='D')
        
        if(len(train[train.index > pd.to_datetime(d,format='%Y%m%d')]) < offset_count):
            print()
            print('NO TRANSACTIONS EXPECTED FROM THIS CUSTOMER ',cust)
            print()
            print('Moving on...')
            print('---------------------------------------------------------------------------------------------------')
            continue
        
        if(len(valid)<3):
            print()
            print('NO TRANSACTIONS EXPECTED FROM THIS CUSTOMER ',cust)
            print()
            print('Moving on...')
            print('---------------------------------------------------------------------------------------------------')
            continue
            
        grp=train1.groupby(train1.index).size().reset_index().rename(columns={0:'records'})
        day=pd.to_datetime(date,format='%Y%m%d')-pd.to_timedelta(5,unit='D')
        grp=grp[grp['Pstng.Date'] >= day]
        count=grp['records'].mean()
        print('Average_count= ',count)
        
        
        model = auto_arima(train, trace=True,error_action='warn',n_iter=10,n_jobs=-1,solver='newton')
        model.fit(train)
        forecast = model.forecast(5)
        if(count == 0):
            count=1
        
        dic.update({cust:forecast*count*c})


        print('----------------------------------------------------------------------------------------------------------')


In [4]:
date=20181101
offset_days=60
offset_count=5
c6765=[1,1,1,1,1]
cash(df,date,offset_days,offset_count,c6765)
out=pd.DataFrame.from_dict(dic,orient='index')
out.to_csv('output_cash.csv')

Average_count=  2.0
Fit ARIMA: order=(2, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=-719.851, BIC=-701.867, Fit time=1.042 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=-725.262, BIC=-719.268, Fit time=0.089 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=-723.786, BIC=-714.795, Fit time=0.096 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=-723.704, BIC=-714.712, Fit time=0.238 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=-723.429, BIC=-711.441, Fit time=0.568 seconds
Total fit time: 2.047 seconds


AttributeError: 'ARIMA' object has no attribute 'forecast'

In [18]:
out

,0,1,2,3,4
100237,0.349765,0.349765,0.349765,0.349765,0.349765
100789,4.312711,4.115460,4.076501,4.071958,4.074897
100985,0.897943,0.897997,0.898051,0.898106,0.898160
